In [4]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from tqdm.notebook import tqdm

import os
import re

lookup_thalia = pd.read_csv("../Dataset/From Other Teams/header_items_lookup_df.csv", lineterminator='\n')
thalia = pd.read_csv("../Dataset/From Other Teams/header_items_df.csv", lineterminator='\n')
processed = pd.read_csv("../Dataset/processedData.csv", sep="|")
book = pd.read_csv("../Dataset/book.csv", sep="|")

In [30]:
thalia2 = thalia.join(lookup_thalia.set_index('headerID')['itemID'], on='headerID')
thalia2.drop(['Unnamed: 0'], axis=1, inplace=True)
thalia2.head(3)

,headerID,title,author,publisher,language,number_pages,recommended_age,release_date,mt_st_,description_,itemID
0,0,and the word became a story,{'the author'},{'books on demand'},{'Englisch'},{304},{''},{Timestamp('2018-04-17 00:00:00')},"['FM', 'FN', 'FL']",at the beginning was the wordand from the word...,5649
1,1,evvai,{'cristina polacchini'},{'lulucom'},{'Italienisch'},{106},{''},{Timestamp('2015-08-01 00:00:00')},['YFB'],amicizia scuola amore alti e bassi in and out ...,77497
2,2,leviathan wakes calibans war abaddons gate no...,{'james s a corey'},{'orbit'},{'Englisch'},{1792},{''},{Timestamp('2019-11-19 00:00:00')},['FLS'],for the first tim the first three novels in co...,56963


In [31]:
thalia2.rename(columns = {'title':'title_thalia'}, inplace=True)
print(thalia2.shape)
thalia2.head(3)

(78021, 11)


,headerID,title_thalia,author,publisher,language,number_pages,recommended_age,release_date,mt_st_,description_,itemID
0,0,and the word became a story,{'the author'},{'books on demand'},{'Englisch'},{304},{''},{Timestamp('2018-04-17 00:00:00')},"['FM', 'FN', 'FL']",at the beginning was the wordand from the word...,5649
1,1,evvai,{'cristina polacchini'},{'lulucom'},{'Italienisch'},{106},{''},{Timestamp('2015-08-01 00:00:00')},['YFB'],amicizia scuola amore alti e bassi in and out ...,77497
2,2,leviathan wakes calibans war abaddons gate no...,{'james s a corey'},{'orbit'},{'Englisch'},{1792},{''},{Timestamp('2019-11-19 00:00:00')},['FLS'],for the first tim the first three novels in co...,56963


In [32]:
processed.head(3)

,itemID,title,titleFound,author_raw,author_api,language,collection,isbn,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,24968,'Que prefieres? libro de juegos para niños: Pr...,¿Que Prefieres? Libro de Juegos para niños: Pr...,Jake Jokester,Jake Jake Jokester,un,1.0,9.781655e+12,LIGHTNING SOURCE INC,NaN,2020-01-02,¡Horas de diversión y unión para toda la famil...,YBG,,NaN,NOT_MATURE,125.0,False
1,69507,'el Canguro Tiene Mamá?: Does a Kangaroo Have ...,"Does a Kangaroo Have a Mother, Too? (Spanish e...",Eric Carle,Eric Carle,un,1.0,9.780060e+12,RAYO,Rayo,2002-03-26,¿Los animales tienen mama? ¡Por supuesto que s...,YFP,YFN,http://books.google.com/books/content?id=PozlG...,NOT_MATURE,32.0,False
2,39693,'n Gotiese Kers Engel (Afrikaanse uitgawe),‘n Gotiese Kers Engel (Afrikaanse Taal - Afrik...,Anna Erishkigal,Anna Erishkigal,un,1.0,9.781943e+12,Seraphim Press,Seraphim Press,2016-12-22,Wenner van die “eFestival of Words Best of the...,FM,,http://books.google.com/books/content?id=VWLGD...,NOT_MATURE,NaN,True


In [33]:
idTemp = list(processed['itemID'])
missingID = []

for i in range(len(idTemp)):
    missingID.append(idTemp[i].split('|'))

In [34]:
list_ID = sum(missingID, [])

In [35]:
thalia3 = thalia2[thalia2['itemID'].isin(list_ID)]
thalia3 = thalia3.reset_index(drop = True)
# thalia3.to_excel("thalia_missingID.xlsx")
print(thalia3.shape)
thalia3.head(3)

(64516, 11)


,headerID,title_thalia,author,publisher,language,number_pages,recommended_age,release_date,mt_st_,description_,itemID
0,0,and the word became a story,{'the author'},{'books on demand'},{'Englisch'},{304},{''},{Timestamp('2018-04-17 00:00:00')},"['FM', 'FN', 'FL']",at the beginning was the wordand from the word...,5649
1,1,evvai,{'cristina polacchini'},{'lulucom'},{'Italienisch'},{106},{''},{Timestamp('2015-08-01 00:00:00')},['YFB'],amicizia scuola amore alti e bassi in and out ...,77497
2,3,the ultimate vehicle colouring book for kids,{'chetna '},{'westland publications limited'},{''},{104},{''},{Timestamp('2020-08-17 00:00:00')},['YBG'],NaN,24125


In [36]:
thalia4 = thalia3.join(book.set_index('itemID')['title'], on='itemID')

In [37]:
thalia4 = thalia4.sort_values('title')
thalia4 = thalia4.reset_index(drop = True)
thalia4.head(3)

,headerID,title_thalia,author,publisher,language,number_pages,recommended_age,release_date,mt_st_,description_,itemID,title
0,6,tripion minaccia dallo spazio,{'stefano grimaldi'},{'lulucom'},{'Italienisch'},{58},{''},{Timestamp('2014-07-06 00:00:00')},['FL'],il genere umano e destinato a scomparirequesto...,26966,TRIPION minaccia dallo spazio
1,3,the ultimate vehicle colouring book for kids,{'chetna '},{'westland publications limited'},{''},{104},{''},{Timestamp('2020-08-17 00:00:00')},['YBG'],NaN,24125,The Ultimate Vehicle Colouring Book for Kids
2,350,4 happy birthday mallory,{'laurie friedman'},{'darby creek'},{''},{''},{''},{NaT},"['YF', '5HKA']",NaN,64745,"#4 Happy Birthday, Mallory!"


In [38]:
processed = processed.sort_values('title')
processed = processed.reset_index(drop = True)
processed.head(3)

,itemID,title,titleFound,author_raw,author_api,language,collection,isbn,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",Stefano Grimaldi,Stefano Grimaldi,it,2.0,9.781292e+12,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False
1,24125,The Ultimate Vehicle Colouring Book for Kids,""" The Ultimate Vehicle Colouring Book for Kids""",Chetna .,Chetna .,un,2.0,9.789390e+12,Westland Publications Limited,NaN,2020-08-31,NaN,YBG,,NaN,NOT_MATURE,104.0,False
2,64745,"#4 Happy Birthday, Mallory!","Happy Birthday, Mallory!",Laurie Friedman,Laurie Friedman,en,2.0,9.781512e+12,DARBY CREEK,Millbrook Press,2017-01-01,Audisee® eBooks with Audio combine professiona...,YF,5HKA,http://books.google.com/books/content?id=ZkrBD...,NOT_MATURE,160.0,True


In [39]:
processed['title'].iloc[0]

' TRIPION  minaccia dallo spazio'

In [40]:
processed[['title_thalia', 'author_thalia', 'language_thalia']] = ''
# processed['author_thalia'] = 

In [41]:
str(thalia4['title'].iloc[0]) == str(processed['title'].iloc[0])

True

In [42]:
str(thalia4['title'].iloc[0]) == str(processed['title'].iloc[6679])

False

In [43]:
str(thalia4['title'].iloc[11])

'#cincodemayo 110 (Edizione Italiana)'

In [44]:
# i_length= len(processed)
# i_start = 0
# i_end = 0

# for j in tqdm(range(len(thalia4))):

#     for i in range(i_start, i_length):
#         print("j,i:",j,",",i)
#         try:
#             if str(thalia4['title'].iloc[j]) == str(processed['title'].iloc[i]):
#                 print(j,i,thalia4['title'].iloc[j],processed['title'].iloc[i])
#                 processed['title_thalia'].iat[i] = thalia4['title_thalia'].iloc[j]
#                 processed['author_thalia'].iat[i] = thalia4['author'].iloc[j]
#                 processed['language_thalia'].iat[i] = thalia4['language'].iloc[j]
# #                 processed['author_thalia'].iat[i] = re.sub("\{|\}|\'", "", thalia4['author'].iloc[j])
# #                 processed['language_thalia'].iat[i] = re.sub("\{|\}|\'", "", thalia4['language'].iloc[j])
#                 i_start = i-10
#                 if(i_start < 0 ):
#                     i_start = 0
#                 print('i_start:', i_start)   
                
#                 break;                

#         except:
#             break;


i_length= len(thalia4)
i_start = 0
i_end = 0
            
for j in tqdm(range(len(processed))):

    for i in range(i_start, i_length):
#         print("j,i:",j,",",i)
        try:
            if str(thalia4['title'].iloc[i]) == str(processed['title'].iloc[j]):
#                 print(j,i,thalia4['title'].iloc[i],processed['title'].iloc[j])
                processed['title_thalia'].iat[j] = thalia4['title_thalia'].iloc[i]
                processed['author_thalia'].iat[j] = thalia4['author'].iloc[i]
                processed['language_thalia'].iat[j] = thalia4['language'].iloc[i]
#                 processed['author_thalia'].iat[i] = re.sub("\{|\}|\'", "", thalia4['author'].iloc[j])
#                 processed['language_thalia'].iat[i] = re.sub("\{|\}|\'", "", thalia4['language'].iloc[j])
                i_start = i-10
                if(i_start < 0 ):
                    i_start = 0
#                 print('i_start:', i_start)   
                
                break;                

        except:
            break;            


In [45]:
processed.head(3)

,itemID,title,titleFound,author_raw,author_api,language,collection,isbn,publisher_raw,publisher_api,...,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook,title_thalia,author_thalia,language_thalia
0,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",Stefano Grimaldi,Stefano Grimaldi,it,2.0,9.781292e+12,Lulu.com,Lulu.com,...,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False,tripion minaccia dallo spazio,{'stefano grimaldi'},{'Italienisch'}
1,24125,The Ultimate Vehicle Colouring Book for Kids,""" The Ultimate Vehicle Colouring Book for Kids""",Chetna .,Chetna .,un,2.0,9.789390e+12,Westland Publications Limited,NaN,...,NaN,YBG,,NaN,NOT_MATURE,104.0,False,the ultimate vehicle colouring book for kids,{'chetna '},{''}
2,64745,"#4 Happy Birthday, Mallory!","Happy Birthday, Mallory!",Laurie Friedman,Laurie Friedman,en,2.0,9.781512e+12,DARBY CREEK,Millbrook Press,...,Audisee® eBooks with Audio combine professiona...,YF,5HKA,http://books.google.com/books/content?id=ZkrBD...,NOT_MATURE,160.0,True,4 happy birthday mallory,{'laurie friedman'},{''}


In [46]:
processed[processed['itemID'] == '5649']

,itemID,title,titleFound,author_raw,author_api,language,collection,isbn,publisher_raw,publisher_api,...,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook,title_thalia,author_thalia,language_thalia
59,5649,... and the Word became a Story,... and the Word became a Story,The Author,The Author,en,2.0,9.783743e+12,Books on Demand,BoD – Books on Demand,...,At the beginning was the Word. And from the Wo...,FL,FL FM FN,http://books.google.com/books/content?id=FrElD...,NOT_MATURE,304.0,False,and the word became a story,{'the author'},{'Englisch'}


In [47]:
processed.isnull().sum()

itemID                 0
title                  0
titleFound             0
author_raw          2361
author_api          2805
language               0
collection             0
isbn                2294
publisher_raw          6
publisher_api      22354
publishedDate        570
description        13705
main topic           201
subtopics              0
thumbnail          11331
maturityRating         0
pageCount           5457
isEbook                0
title_thalia           0
author_thalia          0
language_thalia        0
dtype: int64

In [89]:
processed1 = processed.copy()

In [90]:
str(processed1['author_thalia'].iloc[0])

"{'stefano grimaldi'}"

In [91]:
(re.sub("\{|\}|\'", "", str(processed1['author_thalia'].iloc[0])))

'stefano grimaldi'

In [92]:
for i in range(len(processed1)):
    processed1['author_thalia'].iat[i] = (re.sub("\{|\}|\'", "", str(processed1['author_thalia'].iloc[i])))
    processed1['language_thalia'].iat[i] = (re.sub("\{|\}|\'", "", str(processed1['language_thalia'].iloc[i])))

In [93]:
processed1[processed1['itemID'] == '5649']

,itemID,title,titleFound,author_raw,author_api,language,collection,isbn,publisher_raw,publisher_api,...,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook,title_thalia,author_thalia,language_thalia
59,5649,... and the Word became a Story,... and the Word became a Story,The Author,The Author,en,2.0,9.783743e+12,Books on Demand,BoD – Books on Demand,...,At the beginning was the Word. And from the Wo...,FL,FL FM FN,http://books.google.com/books/content?id=FrElD...,NOT_MATURE,304.0,False,and the word became a story,the author,Englisch


In [94]:
processed1['author_api'] = processed1['author_api'].fillna('')

In [95]:
processed1['googlekey'] = ((processed1['titleFound'] + ' '+ processed1['author_api']).astype(str)).str.lower()
processed1['thaliakey'] = ((processed1['title_thalia'] + ' '+ processed1['author_thalia']).astype(str)).str.lower()
processed1['googlekey'] = processed1['googlekey'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
processed1['thaliakey'] = processed1['thaliakey'].str.replace(pat=r'[^\w]', repl=r'', regex=True)

In [ ]:
print(processed1.shape)
processed1

In [116]:
processed2 = DataFrame(range(0),columns=list(processed1.columns))
thresh = 50
for i in range(len(processed1)):
    if len(processed1['thaliakey'].iloc[i]) != 0:
        if (fuzz.token_sort_ratio(processed1.get('googlekey').iloc[i], processed1.get('thaliakey').iloc[i])) < thresh:
            processed2 = processed2.append(processed1.iloc[i])

In [118]:
processed2.to_excel("thalia_google.xlsx")

In [129]:
addThalia = processed1[processed1['thaliakey'] != '']
print(addThalia.shape)
addThalia

(59726, 23)


,itemID,title,titleFound,author_raw,author_api,language,collection,isbn,publisher_raw,publisher_api,...,subtopics,thumbnail,maturityRating,pageCount,isEbook,title_thalia,author_thalia,language_thalia,googlekey,thaliakey
0,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",Stefano Grimaldi,Stefano Grimaldi,it,2.0,9.781292e+12,Lulu.com,Lulu.com,...,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False,tripion minaccia dallo spazio,stefano grimaldi,Italienisch,tripionminacciadallospaziostefanogrimaldi,tripionminacciadallospaziostefanogrimaldi
1,24125,The Ultimate Vehicle Colouring Book for Kids,""" The Ultimate Vehicle Colouring Book for Kids""",Chetna .,Chetna .,un,2.0,9.789390e+12,Westland Publications Limited,NaN,...,,NaN,NOT_MATURE,104.0,False,the ultimate vehicle colouring book for kids,chetna,,theultimatevehiclecolouringbookforkidschetna,theultimatevehiclecolouringbookforkidschetna
2,64745,"#4 Happy Birthday, Mallory!","Happy Birthday, Mallory!",Laurie Friedman,Laurie Friedman,en,2.0,9.781512e+12,DARBY CREEK,Millbrook Press,...,5HKA,http://books.google.com/books/content?id=ZkrBD...,NOT_MATURE,160.0,True,4 happy birthday mallory,laurie friedman,,happybirthdaymallorylauriefriedman,4happybirthdaymallorylauriefriedman
3,7780,#Basteln for Future,#Basteln for Future,"Susanne Pypke, Naturschutzjugend NAJU",Susanne Pypke,un,2.0,9.783772e+12,Frech Verlag GmbH,NaN,...,5AF Y,NaN,NOT_MATURE,123.0,False,basteln for future,susanne pypke naturschutzjugend naju,,bastelnforfuturesusannepypke,bastelnforfuturesusannepypkenaturschutzjugendnaju
4,27230,#CincoDeMayo (Edicion en español),#CincoDeMayo (Edicion en español),I. D. Oro,I. D. Oro,es,2.0,NaN,INDEPENDENTLY PUBLISHED,I. D. Oro,...,,http://books.google.com/books/content?id=jGVLD...,NOT_MATURE,250.0,True,cincodemayo edicion en español,i d oro,,cincodemayoedicionenespañolidoro,cincodemayoedicionenespañolidoro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59953,72631,Übernachtungs-Party,Übernachtungs-Party,NaN,,un,2.0,9.783831e+12,Dorling Kindersley Verlag,NaN,...,5AH 5AJ 5AK 5JA 4CD WBR WBXN WDM WFBW WFJ WFT...,NaN,NOT_MATURE,80.0,False,uebernachtungs party,nan,Deutsch,übernachtungsparty,uebernachtungspartynan
59954,21473,Übernatürliche Irreführung (Sasha Urban Serie 5),Übernatürliche Irreführung (Sasha Urban Serie 5),"Dima Zales, Anna Zaires","Dima Zales, Anna Zaires",un,2.0,9.781631e+12,MOZAIKA LLC,Mozaika LLC,...,,http://books.google.com/books/content?id=8fsgy...,NOT_MATURE,540.0,False,uebernatuerliche irrefuehrung sasha urban serie 5,dima zales anna zaires,Deutsch,übernatürlicheirreführungsashaurbanserie5dimaz...,uebernatuerlicheirrefuehrungsashaurbanserie5di...
59955,35275,Überschneidung zweier untersschiedlicher Welten,Überschneidung zweier untersschiedlicher Welten,WErner Röschl,WErner Röschl,un,2.0,9.783746e+12,Books on Demand,BoD – Books on Demand,...,FD FM,http://books.google.com/books/content?id=IB-gD...,NOT_MATURE,144.0,False,ueberschneidung zweier untersschiedlicher welten,werner roeschl,Deutsch,überschneidungzweieruntersschiedlicherweltenwe...,ueberschneidungzweieruntersschiedlicherweltenw...
59956,50552,Übersinnlicher Retter,Übersinnlicher Retter,Tina Folsom,Tina Folsom,un,2.0,9.781945e+12,Duboce Park Press,Duboce Park Press,...,,http://books.google.com/books/content?id=G0iZx...,NOT_MATURE,248.0,False,uebersinnlicher retter,tina folsom,Deutsch,übersinnlicherrettertinafolsom,uebersinnlicherrettertinafolsom


In [134]:
keys = addThalia['title']
dup = addThalia[keys.isin(keys[keys.duplicated()])]

In [135]:
dup

,itemID,title,titleFound,author_raw,author_api,language,collection,isbn,publisher_raw,publisher_api,...,subtopics,thumbnail,maturityRating,pageCount,isEbook,title_thalia,author_thalia,language_thalia,googlekey,thaliakey
1893,33670,Adventures of Odysseus,The Adventures of Odysseus,Hugh Lupton,Neil Philip,en,1.0,9.780531e+12,Barefoot Books Ltd,Scholastic,...,YFC,http://books.google.com/books/content?id=2DxiP...,NOT_MATURE,72.0,False,adventures of odysseus,"hugh lupton, hugh lupton daniel morden",Englisch,theadventuresofodysseusneilphilip,adventuresofodysseushughluptonhughluptondaniel...
1894,67408,Adventures of Odysseus,The Adventures of Odysseus,"Hugh Lupton, Daniel Morden",Neil Philip,en,1.0,9.780531e+12,Abrams & Chronicle Books,Scholastic,...,YFJ,http://books.google.com/books/content?id=2DxiP...,NOT_MATURE,72.0,False,adventures of odysseus,"hugh lupton, hugh lupton daniel morden",Englisch,theadventuresofodysseusneilphilip,adventuresofodysseushughluptonhughluptondaniel...
3315,27540,Apotheosis,Apotheosis of an Ideal,Ross Friedman,,en,1.0,NaN,Xlibris,NaN,...,,http://books.google.com/books/content?id=jhEUA...,NOT_MATURE,62.0,True,apotheosis,"nicholas crivac, ross friedman",Englisch,apotheosisofanideal,apotheosisnicholascrivacrossfriedman
3316,47537|8683,Apotheosis,Apotheosis of an Ideal,Nicholas Crivac,,en,1.0,NaN,Stark Road Publishing,NaN,...,,http://books.google.com/books/content?id=jhEUA...,NOT_MATURE,62.0,True,apotheosis,"nicholas crivac, ross friedman",Englisch,apotheosisofanideal,apotheosisnicholascrivacrossfriedman
3338,28342,Aprender a contar para preescolar (Diviértete ...,Aprender a contar para preescolar (Diviértete ...,Santiago Garcia,Garcia Santiago,un,1.0,9.781839e+12,Fichas de preescolar,Aprender a contar para preescolar,...,,http://books.google.com/books/content?id=1knAy...,NOT_MATURE,26.0,False,aprender a contar para preescolar diviértete a...,"garcia santiago, santiago garcia",Spanisch,aprenderacontarparapreescolardiviérteteaprendi...,aprenderacontarparapreescolardiviérteteaprendi...
3339,68889|69588,Aprender a contar para preescolar (Diviértete ...,Aprender a contar para preescolar (Diviértete ...,Garcia Santiago,Garcia Santiago,un,1.0,9.781839e+12,Fichas de preescolar,Aprender a contar para preescolar,...,,http://books.google.com/books/content?id=1knAy...,NOT_MATURE,26.0,False,aprender a contar para preescolar diviértete a...,"garcia santiago, santiago garcia",Spanisch,aprenderacontarparapreescolardiviérteteaprendi...,aprenderacontarparapreescolardiviérteteaprendi...
12925,76243,Die Königin des Lichts,Königin des Lichts,Nora Roberts,Nora Roberts,un,1.0,9.783641e+12,Heyne Taschenbuch,Heyne Verlag,...,FB FRD FYT,http://books.google.com/books/content?id=4m_bb...,NOT_MATURE,416.0,True,die koenigin des lichts,"gisela luise till, nora roberts",Deutsch,königindeslichtsnoraroberts,diekoenigindeslichtsgiselaluisetillnoraroberts
12926,59791,Die Königin des Lichts,Königin des Lichts,Gisela Luise Till,Nora Roberts,un,1.0,9.783641e+12,Papierfresserchens MTM-VE,Heyne Verlag,...,5AL FN YFH YFJ,http://books.google.com/books/content?id=4m_bb...,NOT_MATURE,416.0,True,die koenigin des lichts,"gisela luise till, nora roberts",Deutsch,königindeslichtsnoraroberts,diekoenigindeslichtsgiselaluisetillnoraroberts
14280,66029,Dinosaurier,Wissen. Dinosaurier,Lessoni Animati,,un,1.0,9.783831e+12,Lessoni Animati,NaN,...,,NaN,NOT_MATURE,208.0,False,dinosaurier,"joachim mallok johann brandstetter, nan, lesso...",Deutsch,wissendinosaurier,dinosaurierjoachimmallokjohannbrandstetternanl...
14281,8973,Dinosaurier,Wissen. Dinosaurier,"Joachim Mallok, Johann Brandstetter",,un,1.0,9.783831e+12,Carlsen Verlag GmbH,NaN,...,5AC 4CA YBCS YBLL YNNV 3B ...,NaN,NOT_MATURE,208.0,False,dinosaurier,"joachim mallok johann brandstetter, nan, lesso...",Deutsch,wissendinosaurier,dinosaurierjoachimmallokjohannbrandstetternanl...


In [141]:
addThalia['itemID'] = addThalia['itemID'].astype(str)
addThalia['main topic'] = addThalia['main topic'].astype(str)

<ipython-input-141-7dcfce7e8522>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addThalia['itemID'] = addThalia['itemID'].astype(str)
<ipython-input-141-7dcfce7e8522>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addThalia['main topic'] = addThalia['main topic'].astype(str)


In [173]:
addThalia2 = addThalia.groupby('title').agg({'itemID': '|'.join,
                                    'titleFound': 'first',
                                    'title_thalia': 'first',         
                                    'author_raw': 'first',
                                    'author_api': 'first',
                                    'author_thalia': 'first',
                                    'language':'first',
                                    'language_thalia': 'first',
                                    'collection':'first',
                                    'isbn':'first',
                                    'publisher_raw':'first',
                                    'publisher_api':'first',
                                    'publishedDate': 'first',
                                    'description':'first',
                                    'main topic': '|'.join,
                                    'subtopics': '|'.join,
                                    'thumbnail': 'first',
                                    'maturityRating': 'first',
                                    'pageCount': 'first',
                                    'isEbook': 'first',
                                    }).reset_index()

In [174]:
addThalia2.head(3)

,title,itemID,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,TRIPION minaccia dallo spazio,26966,""" TRIPION "" minaccia dallo spazio",tripion minaccia dallo spazio,Stefano Grimaldi,Stefano Grimaldi,stefano grimaldi,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False
1,The Ultimate Vehicle Colouring Book for Kids,24125,""" The Ultimate Vehicle Colouring Book for Kids""",the ultimate vehicle colouring book for kids,Chetna .,Chetna .,chetna,un,,2.0,...,Westland Publications Limited,NaN,2020-08-31,NaN,YBG,,NaN,NOT_MATURE,104.0,False
2,"#4 Happy Birthday, Mallory!",64745,"Happy Birthday, Mallory!",4 happy birthday mallory,Laurie Friedman,Laurie Friedman,laurie friedman,en,,2.0,...,DARBY CREEK,Millbrook Press,2017-01-01,Audisee® eBooks with Audio combine professiona...,YF,5HKA,http://books.google.com/books/content?id=ZkrBD...,NOT_MATURE,160.0,True


In [175]:
addThalia2 = addThalia2[['itemID', 'title', 'titleFound', 'title_thalia', 'author_raw', 'author_api', 'author_thalia', 'language', 'language_thalia', 'collection', 'isbn', 'publisher_raw', 'publisher_api', 'publishedDate', 'description', 'main topic', 'subtopics', 'thumbnail', 'maturityRating', 'pageCount', 'isEbook']]

In [176]:
addThalia2.head(3)

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",tripion minaccia dallo spazio,Stefano Grimaldi,Stefano Grimaldi,stefano grimaldi,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False
1,24125,The Ultimate Vehicle Colouring Book for Kids,""" The Ultimate Vehicle Colouring Book for Kids""",the ultimate vehicle colouring book for kids,Chetna .,Chetna .,chetna,un,,2.0,...,Westland Publications Limited,NaN,2020-08-31,NaN,YBG,,NaN,NOT_MATURE,104.0,False
2,64745,"#4 Happy Birthday, Mallory!","Happy Birthday, Mallory!",4 happy birthday mallory,Laurie Friedman,Laurie Friedman,laurie friedman,en,,2.0,...,DARBY CREEK,Millbrook Press,2017-01-01,Audisee® eBooks with Audio combine professiona...,YF,5HKA,http://books.google.com/books/content?id=ZkrBD...,NOT_MATURE,160.0,True


In [177]:
addThalia3 = addThalia2[addThalia2['language_thalia'] != '']

In [178]:
addThalia3 = addThalia3.reset_index(drop = True)

In [157]:
addThalia3.to_excel("addThalia.xlsx")

In [179]:
addThalia3.iloc[32757]

itemID                                                         51002
title                                              My Hardcover Book
titleFound                                         My Hardcover Book
title_thalia                                                      my
author_raw                                              Siadin Ellis
author_api                                          Marquel Williams
author_thalia                                           siadin ellis
language                                                          un
language_thalia                                    Deutsch, Englisch
collection                                                         2
isbn                                                     9.78139e+12
publisher_raw                                               Lulu.com
publisher_api                                               Lulu.com
publishedDate                                             2017-05-30
description        This is raw emo

In [180]:
addThalia4 = addThalia3.drop([addThalia3.index[32757]])

In [181]:
addThalia4 = addThalia4.reset_index(drop = True)
addThalia4

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",tripion minaccia dallo spazio,Stefano Grimaldi,Stefano Grimaldi,stefano grimaldi,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False
1,32836,#famous,#famous,famous,Jilly Gagnon,Jilly Gagnon,"jilly gagnon, todd strasser",de,", Englisch",2.0,...,HarperCollins Publishers Inc,Planet!,2018-02-13,Eine turbulente Social-Media-Liebesgeschichte ...,YFM,YXQ,http://books.google.com/books/content?id=kDM5D...,NOT_MATURE,368.0,True
2,38332,''Grounded'' at Fort Irwin,''Grounded'' at Fort Irwin,grounded at fort irwin,Keon Fleurima,Keon Fleurima,keon fleurima,un,Englisch,2.0,...,Xlibris,Xlibris Corporation,2011-11-15,Grounded is about me getting in trouble. After...,YNMH,YFB,http://books.google.com/books/content?id=wY-DA...,NOT_MATURE,NaN,False
3,51507,''It's Not Ok'',''It's Not OK'',its not ok,Amy Castle,Amy Castle,amy castle,en,Englisch,2.0,...,Xlibris,Xlibris Corporation,2013-11-11,"Although it's been around forever, bullying ha...",YFB,YXW,http://books.google.com/books/content?id=B8CDA...,NOT_MATURE,32.0,True
4,13238,'... then he ate my boy entrancers.',Then He Ate My Boy Entrancers,then he ate my boy entrancers,Louise Rennison,,louise rennison,un,Englisch,2.0,...,HarperCollins Publishers,NaN,2008,NaN,YFM,4Z GB ACN YFQ,NaN,NOT_MATURE,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57938,35973,Überlebenstraining mit Leo da Vinci,Überlebenstraining mit Leo Da Vinci,ueberlebenstraining mit leo da vinci,NaN,,nan,un,Deutsch,2.0,...,Edition XXL GmbH,NaN,2019-06-03,NaN,YBG,5AH YNH,NaN,NOT_MATURE,64.0,False
57939,72631,Übernachtungs-Party,Übernachtungs-Party,uebernachtungs party,NaN,,nan,un,Deutsch,2.0,...,Dorling Kindersley Verlag,NaN,2020-01-27,NaN,YNPH2,5AH 5AJ 5AK 5JA 4CD WBR WBXN WDM WFBW WFJ WFT...,NaN,NOT_MATURE,80.0,False
57940,21473,Übernatürliche Irreführung (Sasha Urban Serie 5),Übernatürliche Irreführung (Sasha Urban Serie 5),uebernatuerliche irrefuehrung sasha urban serie 5,"Dima Zales, Anna Zaires","Dima Zales, Anna Zaires",dima zales anna zaires,un,Deutsch,2.0,...,MOZAIKA LLC,Mozaika LLC,2019-09-10,Da ich meine Seherkräfte endlich unter Kontrol...,FMX,,http://books.google.com/books/content?id=8fsgy...,NOT_MATURE,540.0,False
57941,35275,Überschneidung zweier untersschiedlicher Welten,Überschneidung zweier untersschiedlicher Welten,ueberschneidung zweier untersschiedlicher welten,WErner Röschl,WErner Röschl,werner roeschl,un,Deutsch,2.0,...,Books on Demand,BoD – Books on Demand,2019-07-02,Wenn du nach getaner Arbeit nachhause kommst u...,FD,FD FM,http://books.google.com/books/content?id=IB-gD...,NOT_MATURE,144.0,False


In [182]:
addThalia4[addThalia4['title'] == 'My Hardcover Book']

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook


In [186]:
addThalia4.to_csv("processedDataWithThaliaAdded.csv", sep="|", index=False)

In [7]:
addThalia = pd.read_csv("../Dataset/processedDataWithThaliaAdded.csv", sep = "|")

In [8]:
addThalia

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",tripion minaccia dallo spazio,Stefano Grimaldi,Stefano Grimaldi,stefano grimaldi,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False
1,32836,#famous,#famous,famous,Jilly Gagnon,Jilly Gagnon,"jilly gagnon, todd strasser",de,", Englisch",2.0,...,HarperCollins Publishers Inc,Planet!,2018-02-13,Eine turbulente Social-Media-Liebesgeschichte ...,YFM,YXQ,http://books.google.com/books/content?id=kDM5D...,NOT_MATURE,368.0,True
2,38332,''Grounded'' at Fort Irwin,''Grounded'' at Fort Irwin,grounded at fort irwin,Keon Fleurima,Keon Fleurima,keon fleurima,un,Englisch,2.0,...,Xlibris,Xlibris Corporation,2011-11-15,Grounded is about me getting in trouble. After...,YNMH,YFB,http://books.google.com/books/content?id=wY-DA...,NOT_MATURE,NaN,False
3,51507,''It's Not Ok'',''It's Not OK'',its not ok,Amy Castle,Amy Castle,amy castle,en,Englisch,2.0,...,Xlibris,Xlibris Corporation,2013-11-11,"Although it's been around forever, bullying ha...",YFB,YXW,http://books.google.com/books/content?id=B8CDA...,NOT_MATURE,32.0,True
4,13238,'... then he ate my boy entrancers.',Then He Ate My Boy Entrancers,then he ate my boy entrancers,Louise Rennison,NaN,louise rennison,un,Englisch,2.0,...,HarperCollins Publishers,NaN,2008,NaN,YFM,4Z GB ACN YFQ,NaN,NOT_MATURE,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57583,35973,Überlebenstraining mit Leo da Vinci,Überlebenstraining mit Leo Da Vinci,ueberlebenstraining mit leo da vinci,NaN,NaN,NaN,un,Deutsch,2.0,...,Edition XXL GmbH,NaN,2019-06-03,NaN,YBG,5AH YNH,NaN,NOT_MATURE,64.0,False
57584,72631,Übernachtungs-Party,Übernachtungs-Party,uebernachtungs party,NaN,NaN,NaN,un,Deutsch,2.0,...,Dorling Kindersley Verlag,NaN,2020-01-27,NaN,YNPH2,5AH 5AJ 5AK 5JA 4CD WBR WBXN WDM WFBW WFJ WFT...,NaN,NOT_MATURE,80.0,False
57585,21473,Übernatürliche Irreführung (Sasha Urban Serie 5),Übernatürliche Irreführung (Sasha Urban Serie 5),uebernatuerliche irrefuehrung sasha urban serie 5,"Dima Zales, Anna Zaires","Dima Zales, Anna Zaires",dima zales anna zaires,un,Deutsch,2.0,...,MOZAIKA LLC,Mozaika LLC,2019-09-10,Da ich meine Seherkräfte endlich unter Kontrol...,FMX,,http://books.google.com/books/content?id=8fsgy...,NOT_MATURE,540.0,False
57586,35275,Überschneidung zweier untersschiedlicher Welten,Überschneidung zweier untersschiedlicher Welten,ueberschneidung zweier untersschiedlicher welten,WErner Röschl,WErner Röschl,werner roeschl,un,Deutsch,2.0,...,Books on Demand,BoD – Books on Demand,2019-07-02,Wenn du nach getaner Arbeit nachhause kommst u...,FD,FD FM,http://books.google.com/books/content?id=IB-gD...,NOT_MATURE,144.0,False


In [10]:
keys = addThalia['title_thalia']
dup_title = addThalia[keys.isin(keys[keys.duplicated()])]
dup_title

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
788,20587,A House Of Pomegranates,A House of Pomegranates,a house of pomegranates,Oscar Wilde,"Oscar Wilde, Jessie Marion King",oscar wilde,en,Deutsch,2.0,...,Lector House,Courier Corporation,2011-09,"Collects four of Wilde's fairy tales, includin...",YFU,,http://books.google.com/books/content?id=eBXUB...,NOT_MATURE,162.0,False
791,61627,A House of Pomegranates,A House of Pomegranates,a house of pomegranates,Oscar Wilde,Oscar Wilde,oscar wilde,un,Deutsch,2.0,...,INDEPENDENTLY PUBLISHED,ReadHowYouWant.com,2008-08-08,"A publication of 1891, A House of Pomegranates...",YFJ,,http://books.google.com/books/content?id=IbSxy...,NOT_MATURE,148.0,False
1512,45256,Ab die Post,Ab die Post,ab die post,Terry Pratchett,Terry Pratchett,"antje von stemm, terry pratchett",un,Deutsch,2.0,...,Goldmann TB,Manhattan,2012-10-30,Der Gauner Feucht von Lipwig wird dazu verdonn...,FBA,FBA FMK FYT,http://books.google.com/books/content?id=F8dkY...,NOT_MATURE,448.0,True
1513,77133,Ab die Post!,Ab die Post!,ab die post,Antje von Stemm,Antje von Stemm,"antje von stemm, terry pratchett",un,Deutsch,2.0,...,Gerstenberg Verlag,NaN,2017-07,In der Zeit von WhatsApp endlich ein Buch darü...,YNL,5AJ YBG,NaN,NOT_MATURE,144.0,False
2103,18137|30298,Ali Baba and the Forty Thieves,Ali Baba and the Forty Thieves,ali baba and the forty thieves,NaN,Collective,"walter crane, nan",un,Deutsch,2.0,...,Capstone Press,Cideb Editrice,2008-01,NaN,YFJ|XADC,| 5AR YFJ ...,http://books.google.com/books/content?id=KnBrQ...,NOT_MATURE,32.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56492,9286,Wolf of the Past: Premium Hardcover Edition,Wolf of the Past: Premium Hardcover Edition,wolf of the past,A. D. McLain,A. D. McLain,"a d mclain, ad mclain",un,Englisch,2.0,...,BLURB INC,Blurb,2021-02-10,"David is a boy without a family, stealing to s...",FRT,FM,http://books.google.com/books/content?id=zsI6z...,NOT_MATURE,284.0,False
56493,22746,Wolf of the Present: Premium Hardcover Edition,Wolf of the Present: Premium Hardcover Edition,wolf of the present,A. D. McLain,A. D. McLain,a d mclain,un,Englisch,2.0,...,BLURB INC,Blurb,2021-02-10,Meg has a troubled past. In and out of foster ...,FRT,FH FM,http://books.google.com/books/content?id=CFkzz...,NOT_MATURE,318.0,False
57293,11687,"Zombies From Space, And Vampires",Zombies from Space and Vampires,zombies from space and vampires,Angela B. Chrysler,Angela B Chrysler,angela b chrysler,un,Englisch,2.0,...,BLURB INC,NaN,2020-12-22,Alien invasion? Zombies? Vampires? Aria is an ...,FL,FM FU,http://books.google.com/books/content?id=U3Mfz...,NOT_MATURE,84.0,False
57299,73778,Zombies from Space and Vampires: Premium Hardc...,Zombies from Space and Vampires,zombies from space and vampires,Angela B. Chrysler,Angela B Chrysler,angela b chrysler,un,Englisch,1.0,...,BLURB INC,NaN,2020-12-22,Alien invasion? Zombies? Vampires? Aria is an ...,FL,FM FU,http://books.google.com/books/content?id=U3Mfz...,NOT_MATURE,84.0,False


In [11]:
dup_title.to_excel("dup_title.xlsx")

In [19]:
addThalia['itemID'] = addThalia['itemID'].astype(str)
addThalia['main topic'] = addThalia['main topic'].astype(str)

addThalia2 = addThalia.groupby('title_thalia').agg({'itemID': '|'.join,
                                    'title': 'first',                 
                                    'titleFound': 'first',       
                                    'author_raw': 'first',
                                    'author_api': 'first',
                                    'author_thalia': 'first',
                                    'language':'first',
                                    'language_thalia': 'first',
                                    'collection':'first',
                                    'isbn':'first',
                                    'publisher_raw':'first',
                                    'publisher_api':'first',
                                    'publishedDate': 'first',
                                    'description':'first',
                                    'main topic': '|'.join,
                                    'subtopics': '|'.join,
                                    'thumbnail': 'first',
                                    'maturityRating': 'first',
                                    'pageCount': 'first',
                                    'isEbook': 'first',
                                    }).reset_index()

In [20]:
addThalia2[addThalia2['title_thalia'] == 'girl out of water']

,title_thalia,itemID,title,titleFound,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
19487,girl out of water,14273|65903,Girl Out of Water,Girl Out of Water,Nat Luurtsema,Nat Luurtsema,"laura silverman, nat luurtsema",un,Englisch,2.0,...,Walker Books Ltd,"Sourcebooks, Inc.",2016-06-02,Fans of Jenny Han and Sarah Dessen will fall i...,YFM|YFB,YFQ YFR | ...,http://books.google.com/books/content?id=Mb2dj...,NOT_MATURE,320.0,False


In [21]:
keys = addThalia2['titleFound']
dup_title = addThalia2[keys.isin(keys[keys.duplicated()])]
dup_title

,title_thalia,itemID,title,titleFound,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
34,100 gute laune raetsel fuer die vorschule,34748,100 Gute-Laune-Rätsel für die Vorschule,100 Gute-Laune-Rätsel für die Vorschule,NaN,NaN,NaN,un,Deutsch,2.0,...,Loewe Verlag GmbH,NaN,2020-02-12,NaN,YPCA2,5AF 4CA WDK WHJ YBGC YBGH YBLC YBLH YNVP YPCA...,NaN,NOT_MATURE,96.0,False
166,2014 ad black eye band ii,76842,2014 A.D. - Black eye (Band II) -,2014 A.D. - Black eye (Band II) -,Harald Kaup,Harald Kaup,harald kaup,un,Deutsch,2.0,...,NOEL-Verlag,NaN,2015-06-30,NaN,FLC,FLC FLS,http://books.google.com/books/content?id=sEMWs...,NOT_MATURE,341.0,False
167,2014 ad black eye band iii,55895,2014 A.D. - Black Eye (Band III),2014 A.D. - Black eye (Band II) -,Harald Kaup,Harald Kaup,harald kaup,un,Deutsch,2.0,...,NOEL-Verlag,NaN,2015-06-30,NaN,FLC,FLS,http://books.google.com/books/content?id=sEMWs...,NOT_MATURE,341.0,False
258,365 years of solitude and sufferings and the r...,54039,365 Years of Solitude and Sufferings and the R...,365 Years of Solitude and Sufferings and the R...,Benjamin Katz,Benjamin Katz,benjamin katz,en,Englisch,2.0,...,Xlibris US,NaN,2019-05-25,There is no available information at this time...,FL,,http://books.google.com/books/content?id=4oJHx...,NOT_MATURE,356.0,False
259,365 years of solitude sufferings and the rise ...,49635,"365 Years of Solitude, Sufferings and the Rise...",365 Years of Solitude and Sufferings and the R...,Benjamin Katz,Benjamin Katz,benjamin katz,un,Englisch,1.0,...,Xlibris US,NaN,2019-05-25,There is no available information at this time...,FL,,http://books.google.com/books/content?id=CcJKx...,NOT_MATURE,356.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57221,zero repeat forever,60471,Zero Repeat Forever,Zero Repeat Forever,G. S. Prendergast,G. S. Prendergast,g s prendergast,en,Englisch,2.0,...,Simon & Schuster Ltd,Simon and Schuster,2017-08-29,The 5th Wave meets Beauty and the Beast in thi...,YFG,5AN YFH YFM,http://books.google.com/books/content?id=SK2uD...,NOT_MATURE,496.0,True
57222,zero repeat forever volume 1,10032,"Zero Repeat Forever, Volume 1",Zero Repeat Forever,G. S. Prendergast,G. S. Prendergast,g s prendergast,un,Englisch,1.0,...,SIMON & SCHUSTER BOOKS YOU,Simon and Schuster,2017-08-29,"Sixteen year-old Raven, injured and still grie...",YFG,,http://books.google.com/books/content?id=vUMyD...,NOT_MATURE,496.0,False
57398,zwischen den welten,68380|5421,Zwischen den Welten,Zwischen den Welten,NaN,Elina Garanca,"charlotte camp, nan",un,Deutsch,2.0,...,Books on Demand,NaN,2019-02-21,NaN,FM|FKW,,NaN,NOT_MATURE,212.0,False
57399,zwischen den welten gestrandet auf p71,15985,Zwischen den Welten - Gestrandet auf P71,Zwischen den Welten,Celeste Ealain,Celeste Ealain,celeste ealain,un,Deutsch,1.0,...,tredition,BookRix,2020-03-02,Fabiennes Traum sah anders aus: ein Neuanfang ...,FB,,http://books.google.com/books/content?id=1E7UD...,NOT_MATURE,520.0,True


In [22]:
dup_title.to_excel("dup_title.xlsx")

In [47]:
title = ["A Tale Dark and Grimm", "Alice's Adventures in Wonderland" , "Konfirmation-Quiz", "The Wizards of Once: Never and Forever", "The Wonderful Adventures of Nils", "Too Close for Comfort", "Trail of Lightning", "Virals", "Warriors: Dawn of the Clans #1: The Sun Trail"]

In [48]:
a = DataFrame(range(0), columns = list(addThalia2.columns))
for i in range(len(title)):
    b = addThalia2[addThalia2['titleFound'] == str(title[i])]
    a = a.append(b)

In [49]:
a

,title_thalia,itemID,title,titleFound,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
1199,a tale dark & grimm,62757,A Tale Dark & Grimm,A Tale Dark and Grimm,Adam Gidwitz,Adam Gidwitz,adam gidwitz,un,Deutsch,1.0,...,PUFFIN BOOKS,Penguin,2010-10-28,In this mischievous and utterly original debut...,YFJ,YFQ,http://books.google.com/books/content?id=0PHmd...,NOT_MATURE,256.0,True
1200,a tale dark and grimm,27604,A Tale Dark and Grimm,A Tale Dark and Grimm,Adam Gidwitz,Adam Gidwitz,adam gidwitz,un,Englisch,2.0,...,Andersen Press Ltd,Random House,2011-09-01,"Reader: beware. Warlocks with dark spells, hun...",YFD,5AK YFJ,http://books.google.com/books/content?id=dXl4Q...,NOT_MATURE,208.0,True
2079,alices adventures in wonderland,58865|44004|18804|33858|38113|76112|18596|3198...,Alice's Adventures in Wonderland,Alice's Adventures in Wonderland,Lewis Carroll,Lewis Carroll,"lewis carroll, lewis carroll tbd, lewis caroll...",un,"Deutsch, Englisch",2.0,...,Pan Macmillan,"Sterling Publishing Company, Inc.",2005,A little girl falls down a rabbit hole and dis...,YBC|YNMF|FMM|YFA|FM|FMB|DCA|YFJ,YFH | ...,http://books.google.com/books/content?id=zEDDS...,NOT_MATURE,129.0,False
2089,alices adventures in wonderland with illustrat...,27165,Alice's Adventures in Wonderland: With Illustr...,Alice's Adventures in Wonderland,Lewis Carroll,Lewis Carroll,lewis carroll,un,Englisch,1.0,...,DOVER PUBN INC,Courier Corporation,2013-03-21,This new edition of Alice's adventures offers ...,YFA,,http://books.google.com/books/content?id=v67DA...,NOT_MATURE,192.0,True
2092,alicess adventures in wonderland,58170,Alices's Adventures in Wonderland,Alice's Adventures in Wonderland,Lewis Carroll,Lewis Carroll,lewis carroll,un,Englisch,2.0,...,LIGHTNING SOURCE INC,NaN,2015-04-02,"On an ordinary summer's afternoon, Alice tumbl...",YFA,,http://books.google.com/books/content?id=0UO5o...,NOT_MATURE,192.0,False
25561,konfirmation quiz,78819,Konfirmation-Quiz,Konfirmation-Quiz,NaN,NaN,NaN,un,Deutsch,2.0,...,Kaufmann Ernst Vlg GmbH,NaN,2018-01-15,Konfirmations-Quiz: Das perfekte Geschenk zur ...,YNRM,5AL 5HKM 5HKQ YBL YZG,NaN,NOT_MATURE,128.0,False
25562,konfirmations quiz,42835,Konfirmations-Quiz,Konfirmation-Quiz,Ute Braun,NaN,ute braun,un,Deutsch,2.0,...,Kaufmann Ernst Vlg GmbH,NaN,2018-01-15,Konfirmations-Quiz: Das perfekte Geschenk zur ...,WDKX,5ABH 5AN 5HKM 5HKQ YBL YNR YNVP YZG ...,NaN,NOT_MATURE,128.0,False
51702,the wizards of once 04 never and forever,32084,The Wizards of Once 04: Never and Forever,The Wizards of Once: Never and Forever,Cressida Cowell,Cressida Cowell,cressida cowell,un,Englisch,2.0,...,Hachette Children's Book,Hodder Children's Books,2020-09-17,The final book in the Wizards of Once quartet....,YFC,5AK 4Z GB ACJ YFD YFH YFP ...,http://books.google.com/books/content?id=IAlLz...,NOT_MATURE,384.0,False
51705,the wizards of once 4 never and forever,16777,The Wizards of Once 4: Never and Forever,The Wizards of Once: Never and Forever,Cressida Cowell,Cressida Cowell,cressida cowell,un,Englisch,2.0,...,Hachette Children's Book,Hachette UK,2020-09-17,The No.1 bestselling series from the author of...,YFC,5AK 4Z GB ACJ YFD YFH YFP ...,http://books.google.com/books/content?id=qcTRD...,NOT_MATURE,384.0,True
51737,the wonderful adventures of nils,60666|26420|48801|60022,The Wonderful Adventures Of Nils,The Wonderful Adventures of Nils,Selma Lagerlof,Selma Lagerlöf,"selma lagerlof, selma lagerloef, selma lagerlã¶f",un,Englisch,2.0,...,"Kessinger Publishing, LLC",NaN,1940,Selfish little Nils learns kindness and wisdom...,YFA|YFA|FRH,| ...,http://books.google.com/books/content?id=TlQNA...,NOT_MATURE,263.0,False


In [51]:
a['itemID'] = a['itemID'].astype(str)
a['main topic'] = a['main topic'].astype(str)

a2 = a.groupby('titleFound').agg({'itemID': '|'.join,
                                    'title': 'first',                 
                                    'title_thalia': 'first',       
                                    'author_raw': 'first',
                                    'author_api': 'first',
                                    'author_thalia': 'first',
                                    'language':'first',
                                    'language_thalia': 'first',
                                    'collection':'first',
                                    'isbn':'first',
                                    'publisher_raw':'first',
                                    'publisher_api':'first',
                                    'publishedDate': 'first',
                                    'description':'first',
                                    'main topic': '|'.join,
                                    'subtopics': '|'.join,
                                    'thumbnail': 'first',
                                    'maturityRating': 'first',
                                    'pageCount': 'first',
                                    'isEbook': 'first',
                                    }).reset_index()

In [52]:
a2

,titleFound,itemID,title,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,A Tale Dark and Grimm,62757|27604,A Tale Dark & Grimm,a tale dark & grimm,Adam Gidwitz,Adam Gidwitz,adam gidwitz,un,Deutsch,1.0,...,PUFFIN BOOKS,Penguin,2010-10-28,In this mischievous and utterly original debut...,YFJ|YFD,YFQ | 5AK YFJ ...,http://books.google.com/books/content?id=0PHmd...,NOT_MATURE,256.0,True
1,Alice's Adventures in Wonderland,58865|44004|18804|33858|38113|76112|18596|3198...,Alice's Adventures in Wonderland,alices adventures in wonderland,Lewis Carroll,Lewis Carroll,"lewis carroll, lewis carroll tbd, lewis caroll...",un,"Deutsch, Englisch",2.0,...,Pan Macmillan,"Sterling Publishing Company, Inc.",2005,A little girl falls down a rabbit hole and dis...,YBC|YNMF|FMM|YFA|FM|FMB|DCA|YFJ|YFA|YFA,YFH | ...,http://books.google.com/books/content?id=zEDDS...,NOT_MATURE,129.0,False
2,Konfirmation-Quiz,78819|42835,Konfirmation-Quiz,konfirmation quiz,Ute Braun,NaN,ute braun,un,Deutsch,2.0,...,Kaufmann Ernst Vlg GmbH,NaN,2018-01-15,Konfirmations-Quiz: Das perfekte Geschenk zur ...,YNRM|WDKX,5AL 5HKM 5HKQ YBL YZG ...,NaN,NOT_MATURE,128.0,False
3,The Wizards of Once: Never and Forever,32084|16777,The Wizards of Once 04: Never and Forever,the wizards of once 04 never and forever,Cressida Cowell,Cressida Cowell,cressida cowell,un,Englisch,2.0,...,Hachette Children's Book,Hodder Children's Books,2020-09-17,The final book in the Wizards of Once quartet....,YFC|YFC,5AK 4Z GB ACJ YFD YFH YFP ...,http://books.google.com/books/content?id=IAlLz...,NOT_MATURE,384.0,False
4,The Wonderful Adventures of Nils,60666|26420|48801|60022|39879,The Wonderful Adventures Of Nils,the wonderful adventures of nils,Selma Lagerlof,Selma Lagerlöf,"selma lagerlof, selma lagerloef, selma lagerlã¶f",un,Englisch,2.0,...,"Kessinger Publishing, LLC",Courier Corporation,1940,Selfish little Nils learns kindness and wisdom...,YFA|YFA|FRH|YFA,| ...,http://books.google.com/books/content?id=TlQNA...,NOT_MATURE,263.0,False
5,Too Close for Comfort,33196|60973,TOO CLOSE FOR COMFORT,too close for comfort,J. A. Rost,Maude Barlow,j a rost,un,Englisch,2.0,...,Indy Pub,McClelland & Stewart Limited,2005,Canada’s greatest advocate considers our place...,FRM|YFCF,5PGM FW | ...,http://books.google.com/books/content?id=NVb0t...,NOT_MATURE,300.0,False
6,Trail of Lightning,45609|1785,Trail of Lightning,trail of lightning,Rebecca Roanhorse,Rebecca Roanhorse,rebecca roanhorse,en,Englisch,2.0,...,Simon + Schuster Inc.,Hachette UK,2019-11-28,"2019 LOCUS AWARD WINNER, BEST FIRST NOVEL 2019...",FLQ|FMX,FB FJ FMX | FLQ ...,http://books.google.com/books/content?id=SzGsD...,NOT_MATURE,304.0,True
7,Virals,42092|20617,Virals,virals,Kathy Reichs,Kathy Reichs,kathy reichs,un,Deutsch,2.0,...,Cornerstone,Random House,2011-05-12,Fourteen-year-old Tory Brennan is as fascinate...,YFB|YNNJ21,| 5AL 5AN YFB ...,http://books.google.com/books/content?id=tBLwS...,NOT_MATURE,464.0,True
8,Warriors: Dawn of the Clans #1: The Sun Trail,70467|16649,Warriors: Dawn of the Clans 01: The Sun Trail,warriors dawn of the clans 01 the sun trail,Erin Hunter,Erin Hunter,erin hunter,un,Englisch,2.0,...,Harper Collins Publ. USA,Harper Collins,2013-03-05,Discover the origins of the warrior Clans in t...,YFC|YFC,YFG YFH YFP | YFH...,http://books.google.com/books/content?id=uycc2...,NOT_MATURE,368.0,True


In [57]:
dup_title2 = dup_title.copy()

In [59]:
dup_title2.loc[1199]

title_thalia                                     a tale dark & grimm
itemID                                                         62757
title                                            A Tale Dark & Grimm
titleFound                                     A Tale Dark and Grimm
author_raw                                              Adam Gidwitz
author_api                                              Adam Gidwitz
author_thalia                                           adam gidwitz
language                                                          un
language_thalia                                              Deutsch
collection                                                         1
isbn                                                      9.7811e+12
publisher_raw                                           PUFFIN BOOKS
publisher_api                                                Penguin
publishedDate                                             2010-10-28
description        In this mischie

In [65]:
dup_title2 = dup_title2.drop(index=1199, axis=0)

In [68]:
dup_title2 = dup_title2.drop(index=[1200, 2079, 2089, 2092, 25561, 25562, 51702, 51705, 51737, 51738, 52775, 52776, 52955, 52956, 54417, 54418, 54917, 54918], axis=0)

In [70]:
# merged = pd.concat([dup_title2, a2])
merged
merged = merged[['itemID', 'title', 'titleFound', 'title_thalia', 'author_raw', 'author_api', 'author_thalia', 'language', 'language_thalia', 'collection', 'isbn', 'publisher_raw', 'publisher_api', 'publishedDate', 'description', 'main topic', 'subtopics', 'thumbnail', 'maturityRating', 'pageCount', 'isEbook']]

In [71]:
merged

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
34,34748,100 Gute-Laune-Rätsel für die Vorschule,100 Gute-Laune-Rätsel für die Vorschule,100 gute laune raetsel fuer die vorschule,NaN,NaN,NaN,un,Deutsch,2.0,...,Loewe Verlag GmbH,NaN,2020-02-12,NaN,YPCA2,5AF 4CA WDK WHJ YBGC YBGH YBLC YBLH YNVP YPCA...,NaN,NOT_MATURE,96.0,False
166,76842,2014 A.D. - Black eye (Band II) -,2014 A.D. - Black eye (Band II) -,2014 ad black eye band ii,Harald Kaup,Harald Kaup,harald kaup,un,Deutsch,2.0,...,NOEL-Verlag,NaN,2015-06-30,NaN,FLC,FLC FLS,http://books.google.com/books/content?id=sEMWs...,NOT_MATURE,341.0,False
167,55895,2014 A.D. - Black Eye (Band III),2014 A.D. - Black eye (Band II) -,2014 ad black eye band iii,Harald Kaup,Harald Kaup,harald kaup,un,Deutsch,2.0,...,NOEL-Verlag,NaN,2015-06-30,NaN,FLC,FLS,http://books.google.com/books/content?id=sEMWs...,NOT_MATURE,341.0,False
258,54039,365 Years of Solitude and Sufferings and the R...,365 Years of Solitude and Sufferings and the R...,365 years of solitude and sufferings and the r...,Benjamin Katz,Benjamin Katz,benjamin katz,en,Englisch,2.0,...,Xlibris US,NaN,2019-05-25,There is no available information at this time...,FL,,http://books.google.com/books/content?id=4oJHx...,NOT_MATURE,356.0,False
259,49635,"365 Years of Solitude, Sufferings and the Rise...",365 Years of Solitude and Sufferings and the R...,365 years of solitude sufferings and the rise ...,Benjamin Katz,Benjamin Katz,benjamin katz,un,Englisch,1.0,...,Xlibris US,NaN,2019-05-25,There is no available information at this time...,FL,,http://books.google.com/books/content?id=CcJKx...,NOT_MATURE,356.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,60666|26420|48801|60022|39879,The Wonderful Adventures Of Nils,The Wonderful Adventures of Nils,the wonderful adventures of nils,Selma Lagerlof,Selma Lagerlöf,"selma lagerlof, selma lagerloef, selma lagerlã¶f",un,Englisch,2.0,...,"Kessinger Publishing, LLC",Courier Corporation,1940,Selfish little Nils learns kindness and wisdom...,YFA|YFA|FRH|YFA,| ...,http://books.google.com/books/content?id=TlQNA...,NOT_MATURE,263.0,False
5,33196|60973,TOO CLOSE FOR COMFORT,Too Close for Comfort,too close for comfort,J. A. Rost,Maude Barlow,j a rost,un,Englisch,2.0,...,Indy Pub,McClelland & Stewart Limited,2005,Canada’s greatest advocate considers our place...,FRM|YFCF,5PGM FW | ...,http://books.google.com/books/content?id=NVb0t...,NOT_MATURE,300.0,False
6,45609|1785,Trail of Lightning,Trail of Lightning,trail of lightning,Rebecca Roanhorse,Rebecca Roanhorse,rebecca roanhorse,en,Englisch,2.0,...,Simon + Schuster Inc.,Hachette UK,2019-11-28,"2019 LOCUS AWARD WINNER, BEST FIRST NOVEL 2019...",FLQ|FMX,FB FJ FMX | FLQ ...,http://books.google.com/books/content?id=SzGsD...,NOT_MATURE,304.0,True
7,42092|20617,Virals,Virals,virals,Kathy Reichs,Kathy Reichs,kathy reichs,un,Deutsch,2.0,...,Cornerstone,Random House,2011-05-12,Fourteen-year-old Tory Brennan is as fascinate...,YFB|YNNJ21,| 5AL 5AN YFB ...,http://books.google.com/books/content?id=tBLwS...,NOT_MATURE,464.0,True


In [78]:
notdup = addThalia2[~keys.isin(keys[keys.duplicated()])]

In [79]:
notdup = notdup[['itemID', 'title', 'titleFound', 'title_thalia', 'author_raw', 'author_api', 'author_thalia', 'language', 'language_thalia', 'collection', 'isbn', 'publisher_raw', 'publisher_api', 'publishedDate', 'description', 'main topic', 'subtopics', 'thumbnail', 'maturityRating', 'pageCount', 'isEbook']]
notdup

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,5649,... and the Word became a Story,... and the Word became a Story,and the word became a story,The Author,The Author,the author,en,Englisch,2.0,...,Books on Demand,BoD – Books on Demand,2018-04-17,At the beginning was the Word. And from the Wo...,FL,FL FM FN,http://books.google.com/books/content?id=FrElD...,NOT_MATURE,304.0,False
1,77497,... EVVAI!,... EVVAI!,evvai,Cristina Polacchini,CRISTINA POLACCHINI,cristina polacchini,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2015-08-01,"Amicizia, scuola, amore, alti e bassi, in and ...",YFB,,http://books.google.com/books/content?id=lGNgC...,NOT_MATURE,106.0,False
2,13238,'... then he ate my boy entrancers.',Then He Ate My Boy Entrancers,then he ate my boy entrancers,Louise Rennison,NaN,louise rennison,un,Englisch,2.0,...,HarperCollins Publishers,NaN,2008,NaN,YFM,4Z GB ACN YFQ,NaN,NOT_MATURE,NaN,False
3,24800,... trägt Jeans und Tennisschuhe,"""... trägt Jeans und Tennisschuhe""",traegt jeans und tennisschuhe,Frauke Kühn,Frauke Kühn,frauke kuehn,un,Deutsch,2.0,...,Rowohlt Repertoire,NaN,1995,NaN,YFCF,5AN 1DFG 1QBDN YFB YFY YXH YXJ 3MPQX ...,NaN,NOT_MATURE,7.0,False
4,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",tripion minaccia dallo spazio,Stefano Grimaldi,Stefano Grimaldi,stefano grimaldi,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57465,59981,Ícaro en el mundo de los sueños,Ícaro en el mundo de los sueños,ícaro en el mundo de los sueños,Jorge Micó Giner,Jorge Micó Giner,jorge micó giner,un,Spanisch,2.0,...,Lulu.com,Lulu.com,2012-10,"Un muchacho perdido en medio de la ciudad, sin...",FL,,http://books.google.com/books/content?id=Cgj1A...,NOT_MATURE,284.0,False
57466,45360,Órbitas Míticas Volume I,Órbitas Míticas Volume I,órbitas míticas volume i,Kerry Nietz,Kerry Nietz,kerry nietz,un,Portugiesisch,2.0,...,Bear Publications,NaN,2020-05-30,Quatorze das melhores histórias de Ficção Espe...,DNT,FM,http://books.google.com/books/content?id=wFyTz...,NOT_MATURE,266.0,False
57467,32069,Óscar el Oso Pardo Aprende a Ser Agradecido,Óscar el Oso Pardo Aprende a Ser Agradecido,óscar el oso pardo aprende a ser agradecido,Misty Black,Misty Black,misty black,un,Spanisch,2.0,...,Berry Patch Press LLC,NaN,2021-04-21,¡La gratitud nos ayuda a ser felices aún cuand...,YNMD,5HCS YFB,http://books.google.com/books/content?id=MZZez...,NOT_MATURE,NaN,False
57468,55191,"Ô, Yrdann 1","Ô, Yrdann 1",ô yrdann 1,Hadrien Sins,Hadrien Sins,hadrien sins,un,Französisch,2.0,...,Books on Demand,BoD - Books on Demand,2020-05-23,Un cadeau des confins de l'espace vient pertur...,FLS,5AX FM,http://books.google.com/books/content?id=0D_nD...,NOT_MATURE,332.0,False


In [80]:
merge = pd.concat([notdup, merged])

In [83]:
merge[merge['title'] == 'A Tale Dark & Grimm']

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,62757|27604,A Tale Dark & Grimm,A Tale Dark and Grimm,a tale dark & grimm,Adam Gidwitz,Adam Gidwitz,adam gidwitz,un,Deutsch,1.0,...,PUFFIN BOOKS,Penguin,2010-10-28,In this mischievous and utterly original debut...,YFJ|YFD,YFQ | 5AK YFJ ...,http://books.google.com/books/content?id=0PHmd...,NOT_MATURE,256.0,True


In [81]:
merge

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,5649,... and the Word became a Story,... and the Word became a Story,and the word became a story,The Author,The Author,the author,en,Englisch,2.0,...,Books on Demand,BoD – Books on Demand,2018-04-17,At the beginning was the Word. And from the Wo...,FL,FL FM FN,http://books.google.com/books/content?id=FrElD...,NOT_MATURE,304.0,False
1,77497,... EVVAI!,... EVVAI!,evvai,Cristina Polacchini,CRISTINA POLACCHINI,cristina polacchini,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2015-08-01,"Amicizia, scuola, amore, alti e bassi, in and ...",YFB,,http://books.google.com/books/content?id=lGNgC...,NOT_MATURE,106.0,False
2,13238,'... then he ate my boy entrancers.',Then He Ate My Boy Entrancers,then he ate my boy entrancers,Louise Rennison,NaN,louise rennison,un,Englisch,2.0,...,HarperCollins Publishers,NaN,2008,NaN,YFM,4Z GB ACN YFQ,NaN,NOT_MATURE,NaN,False
3,24800,... trägt Jeans und Tennisschuhe,"""... trägt Jeans und Tennisschuhe""",traegt jeans und tennisschuhe,Frauke Kühn,Frauke Kühn,frauke kuehn,un,Deutsch,2.0,...,Rowohlt Repertoire,NaN,1995,NaN,YFCF,5AN 1DFG 1QBDN YFB YFY YXH YXJ 3MPQX ...,NaN,NOT_MATURE,7.0,False
4,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",tripion minaccia dallo spazio,Stefano Grimaldi,Stefano Grimaldi,stefano grimaldi,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,60666|26420|48801|60022|39879,The Wonderful Adventures Of Nils,The Wonderful Adventures of Nils,the wonderful adventures of nils,Selma Lagerlof,Selma Lagerlöf,"selma lagerlof, selma lagerloef, selma lagerlã¶f",un,Englisch,2.0,...,"Kessinger Publishing, LLC",Courier Corporation,1940,Selfish little Nils learns kindness and wisdom...,YFA|YFA|FRH|YFA,| ...,http://books.google.com/books/content?id=TlQNA...,NOT_MATURE,263.0,False
5,33196|60973,TOO CLOSE FOR COMFORT,Too Close for Comfort,too close for comfort,J. A. Rost,Maude Barlow,j a rost,un,Englisch,2.0,...,Indy Pub,McClelland & Stewart Limited,2005,Canada’s greatest advocate considers our place...,FRM|YFCF,5PGM FW | ...,http://books.google.com/books/content?id=NVb0t...,NOT_MATURE,300.0,False
6,45609|1785,Trail of Lightning,Trail of Lightning,trail of lightning,Rebecca Roanhorse,Rebecca Roanhorse,rebecca roanhorse,en,Englisch,2.0,...,Simon + Schuster Inc.,Hachette UK,2019-11-28,"2019 LOCUS AWARD WINNER, BEST FIRST NOVEL 2019...",FLQ|FMX,FB FJ FMX | FLQ ...,http://books.google.com/books/content?id=SzGsD...,NOT_MATURE,304.0,True
7,42092|20617,Virals,Virals,virals,Kathy Reichs,Kathy Reichs,kathy reichs,un,Deutsch,2.0,...,Cornerstone,Random House,2011-05-12,Fourteen-year-old Tory Brennan is as fascinate...,YFB|YNNJ21,| 5AL 5AN YFB ...,http://books.google.com/books/content?id=tBLwS...,NOT_MATURE,464.0,True


In [84]:
merge.to_csv("processedDataWithThaliaAdded.csv", sep="|", index=False)